In [1]:
%config IPCompleter.greedy=True

In [2]:
import requests
import altair as alt
import pandas as pd
from pandas.io.json import json_normalize

from pyvis.network import Network

%load_ext autoreload
%autoreload 2
# import visualizations1 as viz
%cd ../intervals
from main import *
from main_objs import *
import visualizations as viz

/Users/dangtrang/OneDrive - brynmawr.edu/summer 2021/crim_intervals/intervals


## Relationships heatmaps

In [3]:
data_relationships = requests.get('http://crimproject.org/data/relationships/').json()
#df = pd.DataFrame(data)
df_relationships = pd.json_normalize(data_relationships)
files = df_relationships['model_observation.piece.piece_id'].unique()
df_relationships_0011 = df_relationships[df_relationships['model_observation.piece.piece_id'] == files[0]].copy()

In [4]:
df_relationships_0011.head()

,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,rt_tm_minv,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
0,http://crimproject.org/data/relationships/1/,1,Quotation,Fuga,True,True,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...",http://crimproject.org/data/observations/2/,2,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6/1,1,1+3,1+3,3,3/@1-4,@1-3,@1-3+@1-4,@1-3+@..."
1,http://crimproject.org/data/relationships/2/,2,Non-mechanical transformation,Fuga,False,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/4/,4,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-8/1,1+4,1+4,4/@3,@1-4+@4,@1+@1-3,@1"
2,http://crimproject.org/data/relationships/3/,3,Non-mechanical transformation,Fuga,False,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9,14-18/1,1,1-2,1-2,1-2,3,3,3-4,3-4,3-4/@3,@...",http://crimproject.org/data/observations/6/,6,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_5,Missa Tota pulchra es: Agnus Dei,"9-12/3-4,2-4,2,2/@4+@1-4,@4+@1-4+@1-3,@1-3,@1"
3,http://crimproject.org/data/relationships/4/,4,Non-mechanical transformation,Fuga,False,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",http://crimproject.org/data/observations/8/,8,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"5-10/1,1+4,1+4,1+4,1+4,4/@3,@1-4+@4,@1-4.5+@1-..."
183,http://crimproject.org/data/relationships/184/,184,Non-mechanical transformation,Imitative duo,False,False,False,False,False,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"1-7,10-16/1,1,1-2,1-2,2,2,2,3,3,3-4,3-4,4,4,4/...",http://crimproject.org/data/observations/368/,368,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"1-6,10-13/1,1,1+3,1+3,3,3,2,2-3,2-3,3/@1-4,@1-..."


In [5]:
df_relationships_0011[df_relationships_0011['relationship_type']== "Quotation, Mechanical transformation"]

,url,id,relationship_type,musical_type,rt_q,rt_q_x,rt_q_monnayage,rt_tm,rt_tm_snd,rt_tm_minv,...,model_observation.piece.url,model_observation.piece.piece_id,model_observation.piece.full_title,model_observation.ema,derivative_observation.url,derivative_observation.id,derivative_observation.piece.url,derivative_observation.piece.piece_id,derivative_observation.piece.full_title,derivative_observation.ema
256,http://crimproject.org/data/relationships/258/,258,"Quotation, Mechanical transformation",Fuga,True,False,True,True,True,False,...,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,"76-77/1+3,1+3/@3-4.5+@4,@1-2+@1-2.5",http://crimproject.org/data/observations/516/,516,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,"85-87/1,1-2,2/@4,@1-3+@2-4.5,@1"


In [6]:
viz.plot_relationship_heatmap(df_relationships_0011, 'model_observation.ema', category1='relationship_type', category0='observer.name', option=1, heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

In [7]:
df_relationships.iloc[3, :]

url                                             http://crimproject.org/data/relationships/4/
id                                                                                         4
relationship_type                                              Non-mechanical transformation
musical_type                                                                            Fuga
rt_q                                                                                   False
rt_q_x                                                                                 False
rt_q_monnayage                                                                         False
rt_tm                                                                                  False
rt_tm_snd                                                                              False
rt_tm_minv                                                                             False
rt_tm_retrograde                                                      

In [8]:
# def preprocess_df(df, ema_column, ):
#     # preprocess the df to NOT have multiple ema/relationships on the same line
#     df = df.copy()
#     df 

def retrieve_notes_and_durations(piece_url, ema):
    # get the piece
    corpus = CorpusBase([piece_url])
    # get the notes and rests
    model = corpus.scores[0]
    nr = model.getNoteRest()
    dur = model.getDuration()
    beat = model.getBeat()
    
    notes_table = pd.concat([nr, dur, beat], axis=1)
    

def calculate_similarity(df, melody_weight=0.5):
    """
    return a new column with the similarity score for the 
    observations
    by default, we evaluated the melody and the duration equaly (both at 0.5).
    however, the users can choose how heavy they want to weight melody in the caclulation
    """
    
    # retrieve notes and duration from ema
    pass

In [9]:
#retrieve_notes_and_durations('https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_3.0/CRIM_Model_0011.mei', '1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,@1-3,@1')

## Observations heatmap

In [10]:
data_observations = requests.get('http://crimproject.org/data/observations/').json()
#df = pd.DataFrame(data)
df_observations = pd.json_normalize(data_observations)

In [11]:
df_observations.head()

,url,id,ema,musical_type,mt_cf,mt_cf_voices,mt_cf_dur,mt_cf_mel,mt_sog,mt_sog_voices,...,remarks,created,updated,curated,observer.url,observer.name,piece.url,piece.piece_id,piece.full_title,piece.mass
0,http://crimproject.org/data/observations/1/,1,"1-6/1,1,1-2,1-2,2,2/@1,@1-3,@1-3+@1,@1-3+@1-3,...",Fuga,False,,False,False,False,,...,Makes up a longer ID,2017-06-19T19:22:54-04:00,2020-11-27T15:11:10.011852-05:00,True,http://crimproject.org/data/people/CRIM_Person...,Ian Lorenz,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,None
1,http://crimproject.org/data/observations/2/,2,"1-6/1,1,1+3,1+3,3,3/@1-4,@1-3,@1-3+@1-4,@1-3+@...",Fuga,False,,False,False,False,,...,Makes up longer ID,2017-06-19T19:22:54-04:00,2020-07-15T11:37:01.876533-04:00,True,http://crimproject.org/data/people/CRIM_Person...,Ian Lorenz,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,CRIM_Mass_0009
2,http://crimproject.org/data/observations/3/,3,"5-9/1,1,1-2,2,2/@3,@1-4,@1-2+@4,@1-4,@1-2",Fuga,False,,False,False,False,,...,,2017-06-19T19:31:29-04:00,2020-08-02T16:14:06.430415-04:00,True,http://crimproject.org/data/people/CRIM_Person...,Ian Lorenz,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,None
3,http://crimproject.org/data/observations/4/,4,"5-8/1,1+4,1+4,4/@3,@1-4+@4,@1+@1-3,@1",Fuga,False,,False,False,False,,...,,2017-06-19T19:31:29-04:00,2020-08-02T16:14:18.639826-04:00,True,http://crimproject.org/data/people/CRIM_Person...,Ian Lorenz,http://crimproject.org/data/pieces/CRIM_Mass_0...,CRIM_Mass_0009_3,Missa Tota pulchra es: Credo,CRIM_Mass_0009
4,http://crimproject.org/data/observations/5/,5,"5-9,14-18/1,1,1-2,1-2,1-2,3,3,3-4,3-4,3-4/@3,@...",Fuga,False,,False,False,False,,...,,2017-06-19T20:36:44-04:00,2020-08-02T16:19:38.187899-04:00,True,http://crimproject.org/data/people/CRIM_Person...,Drew Ivarson,http://crimproject.org/data/pieces/CRIM_Model_...,CRIM_Model_0011,Tota pulchra es,None


In [12]:
df_observations.columns

Index(['url', 'id', 'ema', 'musical_type', 'mt_cf', 'mt_cf_voices',
       'mt_cf_dur', 'mt_cf_mel', 'mt_sog', 'mt_sog_voices', 'mt_sog_dur',
       'mt_sog_mel', 'mt_sog_ostinato', 'mt_sog_periodic', 'mt_csog',
       'mt_csog_voices', 'mt_csog_dur', 'mt_csog_mel', 'mt_cd', 'mt_cd_voices',
       'mt_fg', 'mt_fg_voices', 'mt_fg_int', 'mt_fg_tint', 'mt_fg_periodic',
       'mt_fg_strict', 'mt_fg_flexed', 'mt_fg_sequential', 'mt_fg_inverted',
       'mt_fg_retrograde', 'mt_pe', 'mt_pe_voices', 'mt_pe_int', 'mt_pe_tint',
       'mt_pe_strict', 'mt_pe_flexed', 'mt_pe_flt', 'mt_pe_sequential',
       'mt_pe_added', 'mt_pe_invertible', 'mt_id', 'mt_id_voices', 'mt_id_int',
       'mt_id_tint', 'mt_id_strict', 'mt_id_flexed', 'mt_id_flt',
       'mt_id_invertible', 'mt_nid', 'mt_nid_voices', 'mt_nid_int',
       'mt_nid_tint', 'mt_nid_strict', 'mt_nid_flexed', 'mt_nid_flt',
       'mt_nid_sequential', 'mt_nid_invertible', 'mt_hr', 'mt_hr_voices',
       'mt_hr_simple', 'mt_hr_staggered', 'mt

In [13]:
df_obvservations_0011 = df_observations[df_observations['piece.piece_id'] == 'CRIM_Model_0011'].copy()

In [14]:
viz.plot_relationship_heatmap(df_obvservations_0011, 'ema', category1='musical_type', category0='observer.name', option=1, heat_map_width=800,
                              heat_map_height=300)

alt.VConcatChart(...)

## Ngrams heatmaps

In [15]:
corpus = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
model = corpus.scores[0]
mel = model.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams = model.getNgrams(df=mel, n=4, cell_type=str)

Requesting file from https://crimproject.org/mei/CRIM_Model_0017.mei...
Successfully imported.


In [16]:
mel_ngrams.stack().dropna().value_counts() 

1, 1, 1, 1      44
1, -2, -3, 2    16
1, 2, 1, -3     16
1, 1, 2, 1      15
1, 2, 2, 1      12
                ..
1, 1, -4, 1      1
2, -3, 3, 1      1
5, -5, 5, 1      1
-4, 2, 1, -3     1
-3, 4, -5, 1     1
Length: 417, dtype: int64

In [17]:
mel_ngrams.head()

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,"1, 1, 1, 2","1, 3, -3, 1",NaN,NaN,NaN,NaN
8.0,"1, 1, 2, 1","3, -3, 1, -2",NaN,NaN,NaN,NaN
12.0,"1, 2, 1, 1","-3, 1, -2, -3",NaN,NaN,NaN,NaN
16.0,"2, 1, 1, 1","1, -2, -3, 3",NaN,NaN,NaN,NaN
20.0,"1, 1, 1, 2","-2, -3, 3, 2",NaN,NaN,NaN,NaN


In [18]:
mel_ngrams_tuple = model.getNgrams(df=mel, n=4, cell_type=tuple)
mel_ngrams_list = model.getNgrams(df=mel, n=4, cell_type=list)

In [19]:
mel_ngrams_tuple.head()

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
4.0,"(1, 1, 1, 2)","(1, 3, -3, 1)",NaN,NaN,NaN,NaN
8.0,"(1, 1, 2, 1)","(3, -3, 1, -2)",NaN,NaN,NaN,NaN
12.0,"(1, 2, 1, 1)","(-3, 1, -2, -3)",NaN,NaN,NaN,NaN
16.0,"(2, 1, 1, 1)","(1, -2, -3, 3)",NaN,NaN,NaN,NaN
20.0,"(1, 1, 1, 2)","(-2, -3, 3, 2)",NaN,NaN,NaN,NaN


In [20]:
viz.plot_close_match_heatmap(mel_ngrams, "1, 1, 1, 1", selected_patterns=[], voices=[])

alt.Chart(...)

In [21]:
viz.plot_close_match_heatmap(mel_ngrams_tuple, "1, 2, 1, -3", selected_patterns=[], voices=[])

alt.Chart(...)

In [22]:
viz.plot_close_match_heatmap(mel_ngrams_list, "1, -2, -3, 2", selected_patterns=[], voices=[])

alt.Chart(...)

## Heatmaps with for ngrams!

How could we create a heatmap for ngrams in which the ngrams' durations are desplayed?

In [23]:
nr = model.getNoteRest()
dur = model.getDuration(nr)
nr.reindex_like(dur)
nr_ngrams = model.getNgrams(df=nr, n=15)
dur_ngrams = model.getNgrams(df=dur, n=15)

In [24]:
nr_ngrams

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,NaN,"(G3, G3, A3, B3, C4, G3, G4, F4, E4, F4, D4, C...",NaN,NaN,NaN,NaN
4.0,NaN,"(G3, A3, B3, C4, G3, G4, F4, E4, F4, D4, C4, D...",NaN,NaN,NaN,NaN
7.0,NaN,"(A3, B3, C4, G3, G4, F4, E4, F4, D4, C4, D4, E...",NaN,NaN,NaN,NaN
8.0,NaN,"(B3, C4, G3, G4, F4, E4, F4, D4, C4, D4, E4, F...",NaN,NaN,NaN,NaN
10.0,NaN,"(C4, G3, G4, F4, E4, F4, D4, C4, D4, E4, F4, G...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1430.0,NaN,NaN,NaN,NaN,NaN,"(F3, G3, F3, G3, D3, E3, F3, G3, F3, F3, G3, A..."
1432.0,NaN,NaN,NaN,NaN,NaN,"(G3, F3, G3, D3, E3, F3, G3, F3, F3, G3, A3, D..."
1434.0,NaN,NaN,NaN,NaN,NaN,"(F3, G3, D3, E3, F3, G3, F3, F3, G3, A3, D3, F..."
1444.0,NaN,NaN,NaN,"(D4, D4, D3, E3, F3, G3, A3, D3, E3, F3, G3, A...",NaN,NaN


In [25]:
dur_ngrams

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,"(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(4.0, 2.0, 8.0, 8.0, 8.0, 8.0, 4.0, 3.0, 4.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 2.0, ...","(8.0, 8.0, 8.0, 8.0, 4.0, 2.0, 8.0, 8.0, 4.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 4.0, ..."
4.0,NaN,"(3.0, 1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 2.0, 3.0, ...",NaN,NaN,NaN,NaN
7.0,NaN,"(1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 2.0, 3.0, 1.0, ...",NaN,NaN,NaN,NaN
8.0,"(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(2.0, 8.0, 8.0, 8.0, 8.0, 4.0, 3.0, 4.0, 2.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 2.0, 2.0, ...","(8.0, 8.0, 8.0, 4.0, 2.0, 8.0, 8.0, 4.0, 4.0, ...","(8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 4.0, 2.0, ..."
10.0,NaN,"(2.0, 2.0, 4.0, 1.0, 1.0, 2.0, 3.0, 1.0, 1.0, ...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1462.0,NaN,NaN,NaN,"(2.0, 16.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.3333333...",NaN,NaN
1463.0,NaN,NaN,"(1.0, 2.0, 1.0, 1.0, 4.0, 2.0, 1.0, 4.0, 1.0, ...",NaN,NaN,"(1.0, 2.0, 2.0, 2.0, 4.0, 4.0, 2.0, 2.0, 4.0, ..."
1464.0,NaN,"(8.0, 3.0, 1.0, 2.0, 2.0, 4.0, 1.0, 1.0, 2.0, ...","(1.0, 8.0, 8.0, 8.0, 4.0, 6.0, 8.0, 4.0, 2.0, ...","(4.0, 1.0, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, ...",NaN,"(16.0, 3.0, 2.0, 4.0, 1.0, 1.0, 1.0, 1.0, 3.0,..."
1468.0,NaN,NaN,NaN,"(16.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.333333333333...",NaN,NaN


In [26]:
# convert the patterns to string for better display in the heatmap
nr_ngrams  = nr_ngrams .applymap(lambda cell: ", ".join(item for item in cell) if pd.notna(cell) else cell) 

# convert the tuple to the total duration of corresponding ngrams
dur_ngrams = dur_ngrams.applymap(lambda cell: sum(cell) if pd.notna(cell) else cell)

In [27]:
dur_ngrams 

,[Discantus],[Contratenor],[Tenor],[QuintaVox],[SextaVox],[Bassus]
0.0,108.0,62.0,112.0,90.000000,85.0,89.0
4.0,NaN,25.0,NaN,NaN,NaN,NaN
7.0,NaN,23.0,NaN,NaN,NaN,NaN
8.0,106.0,66.0,112.0,82.666667,85.0,85.0
10.0,NaN,26.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
1462.0,NaN,NaN,NaN,35.333333,NaN,NaN
1463.0,NaN,NaN,28.0,NaN,NaN,41.0
1464.0,NaN,32.0,74.0,33.000000,NaN,46.0
1468.0,NaN,NaN,NaN,34.333333,NaN,NaN


In [28]:
# plot only the most popular patterns
most_popular = nr_ngrams.stack().value_counts().head(20).index.to_list()

In [29]:
viz.plot_ngrams_heatmap(nr_ngrams, dur_ngrams, selected_pattern=most_popular, voices=[])

alt.VConcatChart(...)

In [30]:
# retrieve necessary ngrams
corpus8 = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0008.mei'])
corpus17 = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0017.mei'])
corpus11 = CorpusBase(['https://crimproject.org/mei/CRIM_Model_0011.mei'])

Requesting file from https://crimproject.org/mei/CRIM_Model_0008.mei...
Successfully imported.
Memoized piece detected...
Requesting file from https://crimproject.org/mei/CRIM_Model_0011.mei...
Successfully imported.


In [31]:
model8 = corpus8.scores[0]
mel8 = model8.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams8 = model8.getNgrams(df=mel8, n=4)

model17 = corpus17.scores[0]
mel17 = model17.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams17 = model17.getNgrams(df=mel17, n=4)

model11 = corpus11.scores[0]
mel11 = model11.getMelodic(kind='d', compound=False, unit=4)
mel_ngrams11 = model11.getNgrams(df=mel11, n=4)

In [32]:
mel_ngrams8.index.name = "offset"

mel_ngrams8.reset_index().melt(id_vars=['offset'], value_name='pattern', var_name=['voice'])

,offset,voice,pattern
0,4.0,[Superius],"(4, 1, 1, 2)"
1,8.0,[Superius],"(1, 1, 2, 2)"
2,12.0,[Superius],"(1, 2, 2, -3)"
3,16.0,[Superius],"(2, 2, -3, 1)"
4,20.0,[Superius],NaN
...,...,...,...
1087,1272.0,Bassus,"(-5, 1, 1, 1)"
1088,1276.0,Bassus,"(1, 1, 1, 1)"
1089,1280.0,Bassus,"(1, 1, 1, 1)"
1090,1284.0,Bassus,"(1, 1, 1, 1)"


In [87]:
# add from one model's ngram to the graph
def add_ngram_to_graph(graph, name, ngrams):
    ngrams = ngrams.reset_index().melt(id_vars=['offset'], value_name='pattern', var_name=['voice']).dropna()
#     sub_patterns = ngrams['pattern'].map(lambda ngram: ngram[:i] for i in range(1, len(ngram)))
    # TODO fix for loop!
    for i in range(len(ngrams)):
        ngram = ngrams.iloc[i, :]
        # add nodes
        ngram_sliced = [ngram['pattern'][:i] for i in range(1, len(ngram['pattern']) + 1)]
        ngram_sliced = [", ".join(item for item in cell) for cell in ngram_sliced]
        ngram_sliced.append(name + ' ' + str(ngram['offset']))
        graph.add_nodes(ngram_sliced)
        print(ngram_slic net = Network()ed)
#         # add edges
#         for i in range(len(ngram_sliced) - 1):
#             graph.add_edge(ngram_sliced[i], ngram_sliced[i+1])

In [88]:
net = Network()
add_ngram_to_graph(net, 'CRIM_Model_0008.mei', mel_ngrams8)

['4', '4, 1', '4, 1, 1', '4, 1, 1, 2', 'CRIM_Model_0008.mei 4.0']
['1', '1, 1', '1, 1, 2', '1, 1, 2, 2', 'CRIM_Model_0008.mei 8.0']
['1', '1, 2', '1, 2, 2', '1, 2, 2, -3', 'CRIM_Model_0008.mei 12.0']
['2', '2, 2', '2, 2, -3', '2, 2, -3, 1', 'CRIM_Model_0008.mei 16.0']
['1', '1, -3', '1, -3, -2', '1, -3, -2, 2', 'CRIM_Model_0008.mei 60.0']
['-3', '-3, -2', '-3, -2, 2', '-3, -2, 2, 3', 'CRIM_Model_0008.mei 64.0']
['-2', '-2, 2', '-2, 2, 3', '-2, 2, 3, 1', 'CRIM_Model_0008.mei 68.0']
['2', '2, 3', '2, 3, 1', '2, 3, 1, -2', 'CRIM_Model_0008.mei 72.0']
['3', '3, 1', '3, 1, -2', '3, 1, -2, 2', 'CRIM_Model_0008.mei 76.0']
['1', '1, -2', '1, -2, 2', '1, -2, 2, 1', 'CRIM_Model_0008.mei 80.0']
['1', '1, 1', '1, 1, 2', '1, 1, 2, 2', 'CRIM_Model_0008.mei 128.0']
['1', '1, 2', '1, 2, 2', '1, 2, 2, -3', 'CRIM_Model_0008.mei 132.0']
['2', '2, 2', '2, 2, -3', '2, 2, -3, -3', 'CRIM_Model_0008.mei 136.0']
['2', '2, -3', '2, -3, -3', '2, -3, -3, -3', 'CRIM_Model_0008.mei 140.0']
['-3', '-3, -3', '-3, -3,

In [78]:
net.show('mygraph.html')

In [90]:
net = Network()
nodes = ["a", "b", "c", "d"]
net.add_nodes(nodes)
net.add_edge("a", "b")
net.show('lol.html')